### Prerequisites

In [ ]:
# !pip install torchmetrics

In [3]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms, datasets
from copy import deepcopy
import requests
from PIL import Image
from resnet_cifar import Trainer, cifar_dataloader


def load_img(url):
    IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD = ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    transform = transforms.Compose([
        transforms.Resize(256, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD)
        ])
    if url.startswith("https"):
        img = Image.open(requests.get(url, stream=True).raw)
    else:
        img = Image.open(url)
    img = transform(img).unsqueeze(0)
    return img


def get_predictions(outp):
    cls_idx = {
        0: 'airplane',
        1: 'automobile',
        2: 'bird',
        3: 'cat',
        4: 'deer',
        5: 'dog',
        6: 'frog',
        7: 'horse',
        8: 'ship',
        9: 'truck'}
    outp = F.softmax(outp, dim=1)
    score, idx = torch.topk(outp, 1)
    idx.squeeze_()
    predicted_label = cls_idx[idx.item()]
    print(predicted_label, '(', score.squeeze().item(), ')')


def print_sizeof(model):
    total = 0
    for p in model.parameters():
        total += p.numel() * p.element_size()
    total /= 1e6
    print("Model size: ", total, " MB")


In [4]:
print(torch.__version__)

1.10.2


## Flowchart for using Quantization in PyTorch

<img src="./img/quantization-flowchart.png" width="700" />

## 10M+ Parameters?

<img src="./img/flowchart-check1.png" width="300" />

Quantization works best on models with 10M+ parameters. [[1](https://arxiv.org/pdf/1806.08342.pdf)]

Large models are more robust to quantization error. Overparameterized models generally have more degrees of freedom and can afford the precision drops with quantization.

As with most thumb rules, YMMV. Quantization is an active area of research, and this might become more permissive.

In [5]:
def is_large_enough(model):
    n_params = sum([p.numel() for p in model.parameters()])
    return n_params > 1e7, n_params // 1e6

print("resnet18: ", is_large_enough(models.resnet18()))
print("resnet50: ", is_large_enough(models.resnet50()))
print("mobilenet_large: ", is_large_enough(models.mobilenet_v3_large()))
print()

resnet18:  (True, 11.0)
resnet50:  (True, 25.0)
mobilenet_large:  (False, 5.0)



## FP32-pretrained checkpoint?

<img src="./img/flowchart-check2.png" width="300" />

Quantized inference works best on models that were originally trained in FP32 (like all non-quantized pretrained models in PyTorch (vision, audio and text)). 
Even Quantization-Aware Training (more on this below) uses FP32 arithmetic to train the parameters.

....

In this exercise, we'll use an FP32 Imagenet-pretrained Resnet that is finetuned to CIFAR10.

In [6]:
# load model from checkpoint 

weights = torch.hub.load_state_dict_from_url("https://quantization-workshop.s3.amazonaws.com/resnet50_cifar_weights.pth", map_location="cpu")
resnet = models.resnet50(pretrained=False, num_classes=10)
# weights = torch.hub.load_state_dict_from_url("https://quantization-workshop.s3.amazonaws.com/resnet18_cifar_weights.pth", map_location="cpu")
# resnet = models.resnet18(pretrained=False, num_classes=10)

resnet.load_state_dict(weights)

param_0 = next(iter(resnet.parameters()))
print("Model precision: ", param_0.dtype)

Downloading: "https://quantization-workshop.s3.amazonaws.com/resnet50_cifar_weights.pth" to /Users/subramen/.cache/torch/hub/checkpoints/resnet50_cifar_weights.pth


  0%|          | 0.00/90.1M [00:00<?, ?B/s]

Model precision:  torch.float32


Before we start quantizing the model, we need to switch it to `eval` mode. 

In [7]:
resnet.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Using a supported backend?

<img src="./img/flowchart-check3.png" width="300" />

Backend refers to the hardware-specific kernels that support quantization. This controls the numerics engine that does the integer arithmetic.

`torch.backends.quantized.engine` specifies the backend to be used.

Using an incorrect backend engine for your hardware will result in (much) slower inference.

In [8]:
import platform
chip = platform.processor()

if chip == 'arm':
    backend = 'qnnpack'
elif chip in ['x86_64', 'i386']:
    backend = 'fbgemm'
else:
    raise SystemError("Backend is not supported")

print(f"Using {backend} backend engine for {chip} CPU")

torch.backends.quantized.engine = backend

Using qnnpack backend engine for arm CPU


## Profile FP32 model inference

<img src="./img/flowchart-check4.png" width="300" />

Let's establish a baseline for model size, inference latency and accuracy

In [9]:
import os
    
def print_size_of_model(model):
    torch.jit.script(model).save("temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

def profile(model):
    print_size_of_model(model)
    print("="*20)
    Trainer(model, -1).evaluate(max_batch=30)  # latency + accuracy on CIFAR test set 

In [10]:
print("Resnet FP32 profile:")
profile(resnet)

Resnet FP32 profile:
Size (MB): 94.469753


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar_data/cifar-10-python.tar.gz to ./cifar_data
Files already downloaded and verified
Loss: 0.5838258673747381 
Accuracy: 0.8729166666666667
Time taken (1920 CIFAR test samples): 82.51376509666443


## What are the predominant layers in the model?

<img src="./img/flowchart-check5.png" width="300" />

While dynamic quantization has more overhead than static quantization, some operators (like recurrent layers) aren't supported by static quantization. (See [Operator coverage](https://pytorch.org/docs/stable/quantization.html#:~:text=these%20quantization%20types.-,Operator%20coverage,-varies%20between%20dynamic)).

Knowing which layers are in our model can inform our quantization strategy.


#### Thumb rule

* For recurrent and transformer layers, use Dynamic quantization.
* For linear layers, you can use either Dynamic or Static quantization.
* For everything else, use Static quantization.

In [11]:
def optimal_quant_strategy(model):
    from collections import Counter
    layer_counts = Counter([type(x).__name__ for x in model.modules()])
    print("Model consists of: ", layer_counts)
    
    dyn = [0, 0]
    stat = [0, 0]

    for m in model.modules():
        if hasattr(m, 'weight'):    
            name = type(m).__name__
            params = m.weight.numel()
            if name in ['RNN', 'LSTM', 'GRU', 'LSTMCell', 'RNNCell', 'GRUCell', 'Linear']:
                dyn[0] += 1
                dyn[1] += params
            if 'Conv' in name or name == 'Linear':
                stat[0] += 1
                stat[1] += params
    print()
    print("Dynamic quantization")
    print("====================")
    print(f"Layers: {dyn[0]} || Parameters: {format(dyn[1], 'g')}")
    print()
    print("Static quantization")
    print("====================")
    print(f"Layers: {stat[0]} || Parameters: {format(stat[1], 'g')}")
    

optimal_quant_strategy(resnet)

Model consists of:  Counter({'Conv2d': 53, 'BatchNorm2d': 53, 'ReLU': 17, 'Bottleneck': 16, 'Sequential': 8, 'ResNet': 1, 'MaxPool2d': 1, 'AdaptiveAvgPool2d': 1, 'Linear': 1})

Dynamic quantization
Layers: 1 || Parameters: 20480

Static quantization
Layers: 54 || Parameters: 2.34754e+07


## Try Dynamic Quantization

<img src="./img/flowchart-check6.png" width="300" />

[Dynamic Quantization API](https://pytorch.org/docs/stable/generated/torch.quantization.quantize_dynamic.html?highlight=quantize_dynamic#torch.quantization.quantize_dynamic)

[Dynamic Quantization Tutorial](https://pytorch.org/tutorials/recipes/recipes/dynamic_quantization.html)

In [12]:
from torch.quantization.quantize_fx import prepare_fx, convert_fx

dynamic_qconfig = torch.quantization.default_dynamic_qconfig
qconfig_dict = {
    # Global Config
    "": dynamic_qconfig
}

model_prepared = prepare_fx(resnet, qconfig_dict)
dynamic_resnet = convert_fx(model_prepared)

/opt/miniconda3/lib/python3.9/site-packages/torch/quantization/fx/quantization_patterns.py:616: UserWarning: dtype combination: (torch.float32, torch.qint8, torch.quint8) is not supported by Conv supported dtype combinations are: [(torch.quint8, torch.qint8, None)]
  warnings.warn(
/opt/miniconda3/lib/python3.9/site-packages/torch/quantization/fx/quantization_patterns.py:484: UserWarning: dtype combination: (torch.float32, torch.qint8, torch.quint8) is not supported by <built-in function add> for is_reference=False. Supported non-reference dtype combinations are: [(torch.qint8, torch.qint8, None), (torch.quint8, torch.qint8, None), (torch.float16, torch.float16, None)] 
  warnings.warn(


### Evaluate performance of dynamic-quantized Resnet model

In [13]:
print("Resnet Dynamic-Quant Profile:")
profile(dynamic_resnet)

Resnet Dynamic-Quant Profile:
Size (MB): 94.057089
Files already downloaded and verified
Files already downloaded and verified
Loss: 0.5846356943249702 
Accuracy: 0.8723958333333334
Time taken (1920 CIFAR test samples): 78.73776507377625


## Try Static Quantization

<img src="./img/flowchart-check7_1.png" width="300" />
<br>
<img src="./img/flowchart-check7_2.png" width="300" />

Static quantization has faster inference than dynamic quantization because it eliminates the float<->int conversion costs between layers

### Manual approach - using Eager Mode

Explicitly perform the following steps:

<img src="./img/ptq-flowchart.png" width="300" />

* Manually identify sequence of fusable modules
* Manually insert stubs to quantize and dequantize activations
* Functional ops (eg: `torch.nn.functional.linear`) aren't supported

[Module Fusion Tutorial](https://pytorch.org/tutorials/recipes/fuse.html)

[Static Quantization (Eager Mode) Tutorial](https://pytorch.org/tutorials/advanced/static_quantization_tutorial.html)

### Easier approach - using FX Graph Mode

<img src="./img/ptq-fx-flowchart.png" width="300" />

* Just 2 function calls: `prepare_fx` and `convert_fx`
* Automates all the above steps under the hood using `torch.fx`

[`prepare_fx` API](https://pytorch.org/docs/stable/generated/torch.quantization.quantize_fx.prepare_fx.html#torch.quantization.quantize_fx.prepare_fx)

[`convert_fx` API](https://pytorch.org/docs/stable/generated/torch.quantization.quantize_fx.convert_fx.html#torch.quantization.quantize_fx.convert_fx)

[Static Quantization (FX Graph Mode) Tutorial](https://pytorch.org/tutorials/prototype/fx_graph_mode_ptq_static.html)

#### QConfig

In FX Quantization, the `qconfig_dict` offers fine-grained control of the model's quantization process.
Setting a `qconfig=None` skips quantization for that module.

```python
qconfig_dict = {
    # Global Config
    "": qconfig,

    # Module-specific config (by class)
    "object_type": [
        (torch.nn.Conv2d, qconfig),
        (torch.nn.functional.add, None),  # skips quantization for this module
        ...,
        ],
    
    # Module-specific config (by name)
    "module_name": [
        ("foo.bar", qconfig)
        ...,
    ],
}
```

In [14]:
static_qconfig = torch.quantization.get_default_qconfig(backend)
qconfig_dict = {
    # Global Config
    "": static_qconfig,
}

In [15]:
from torchvision import datasets, transforms
from torch.quantization.quantize_fx import prepare_fx, convert_fx


def static_quantize_vision_model(model, qconfig_dict):
        _, data = cifar_dataloader()
        mp = prepare_fx(model, qconfig_dict)

        for c, (x, y) in enumerate(data):
                if c == 30:
                        break
                mp(x)
        
        mc = convert_fx(mp)
        return mc

static_resnet = static_quantize_vision_model(resnet, qconfig_dict)

Files already downloaded and verified
Files already downloaded and verified


/opt/miniconda3/lib/python3.9/site-packages/torch/ao/quantization/observer.py:886: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_begin // dst_bin_width, 0, self.dst_nbins - 1
/opt/miniconda3/lib/python3.9/site-packages/torch/ao/quantization/observer.py:891: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_end // dst_bin_w

### Evaluate performance of static-quantized model

In [16]:
print("Resnet Static-Quant Profile:")
profile(static_resnet)

Resnet Static-Quant Profile:
Size (MB): 23.661729
Files already downloaded and verified
Files already downloaded and verified
Loss: 0.5700115218758584 
Accuracy: 0.8708333333333333
Time taken (1920 CIFAR test samples): 34.77193212509155


### Sensitivity Analysis - Which quantized layers affect accuracy the most?

<img src="./img/flowchart-check8.png" width="300" />
<br>

Some layers are more sensitive to precision drops than others. PyTorch provides tools to help with this analysis under the Numeric Suite.

[Numeric Suite Tutorial](https://pytorch.org/tutorials/prototype/numeric_suite_tutorial.html)


In [17]:
import torch.quantization._numeric_suite as ns

def SNR(x, y):
    # Higher is better
    Ps = torch.norm(x)
    Pn = torch.norm(x-y)
    return 20 * torch.log10(Ps/Pn)

def compare_model_weights(float_model, quant_model):
    snr_dict = {}
    wt_compare_dict = ns.compare_weights(float_model.state_dict(), quant_model.state_dict())
    for param_name, weight in wt_compare_dict.items():
        snr = SNR(weight['float'], weight['quantized'].dequantize())
        snr_dict[param_name] = snr

    return snr_dict

Layer-by-layer comparison of model weights 

<img src="./img/ns.png" width="300" />

In [18]:
snrd = compare_model_weights(resnet, static_resnet)
print(snrd)

{'conv1.weight': tensor(0.1324), 'layer1.0.conv1.weight': tensor(1.6452), 'layer1.0.conv2.weight': tensor(-2.4947), 'layer1.0.conv3.weight': tensor(-10.0644), 'layer1.0.downsample.0.weight': tensor(1.3075), 'layer1.1.conv1.weight': tensor(3.2131), 'layer1.1.conv2.weight': tensor(1.2505), 'layer1.1.conv3.weight': tensor(-9.6478), 'layer1.2.conv1.weight': tensor(-0.2502), 'layer1.2.conv2.weight': tensor(-6.6366), 'layer1.2.conv3.weight': tensor(-12.0677), 'layer2.0.conv1.weight': tensor(-0.1468), 'layer2.0.conv2.weight': tensor(1.0737), 'layer2.0.conv3.weight': tensor(-9.7258), 'layer2.0.downsample.0.weight': tensor(-0.3786), 'layer2.1.conv1.weight': tensor(6.1034), 'layer2.1.conv2.weight': tensor(-3.3722), 'layer2.1.conv3.weight': tensor(-13.6104), 'layer2.2.conv1.weight': tensor(5.4238), 'layer2.2.conv2.weight': tensor(-5.2156), 'layer2.2.conv3.weight': tensor(-13.1915), 'layer2.3.conv1.weight': tensor(3.8682), 'layer2.3.conv2.weight': tensor(-1.7579), 'layer2.3.conv3.weight': tensor(-

In [19]:
def topk_sensitive_layers(snr_dict, k):
    snr_dict = dict(sorted(snr_dict.items(), key=lambda x:x[1]))
    snr_dict = {k.replace('.weight', ''):v for k,v in list(snr_dict.items())[:k]}
    return snr_dict
    
sensitive_layers = topk_sensitive_layers(snrd, 5).keys()
print(sensitive_layers)

dict_keys(['layer4.2.conv3', 'layer4.1.conv3', 'layer4.0.conv3', 'layer4.1.conv2', 'layer4.2.conv2'])


## Selective Static Quantization

In [20]:
sensitive_layers = topk_sensitive_layers(snrd, 5).keys()

qconfig_dict = {
    # Global Config
    "": static_qconfig,

    # Disable for sensitive modules
    "module_name": [(m, None) for m in sensitive_layers],
}

sel_static_resnet = static_quantize_vision_model(resnet, qconfig_dict)

Files already downloaded and verified
Files already downloaded and verified


### Evaluate performance of selective static-quantized model

In [21]:
print("Resnet Selective-Static-Quant Profile:")
profile(sel_static_resnet)

Resnet Selective-Static-Quant Profile:
Size (MB): 47.265735
Files already downloaded and verified
Files already downloaded and verified
Loss: 0.5818707610170046 
Accuracy: 0.8703125
Time taken (1920 CIFAR test samples): 42.71668791770935


## Quantization-Aware Training

<img src="./img/flowchart-check9.png" width="300" />

In [22]:
from torch.quantization.quantize_fx import prepare_qat_fx
from resnet_cifar import Trainer, cifar_dataloader

sensitive_layers = topk_sensitive_layers(snrd, 5).keys()

qat_qconfig = torch.quantization.get_default_qat_qconfig(backend)
qconfig_dict = {
    # Global Config
    "": qat_qconfig,
}

def qat_vision_model(model, qconfig):
    model.train()
    mp = prepare_qat_fx(model, qconfig)

    # training loop
    trainer = Trainer(mp, epochs=120, device='cuda')  
    trainer.run_epoch()
    mp = mp.cpu()

    mc = convert_fx(mp)
    return mc

Training this for 120 epochs takes about 2 hours on a single Tesla V100 GPU.
If you don't want to wait, download the QAT weights instead

In [24]:

# qat_resnet = qat_vision_model(resnet, qconfig_dict) 
# torch.jit.script(qat_resnet).save('qat_resnet50_cifar.pt')

import requests
r = requests.get("https://quantization-workshop.s3.amazonaws.com/qat_resnet50_cifar.pt")
open('qat_resnet50_cifar.pt', 'wb').write(r.content)

qat_resnet = torch.load("qat_resnet50_cifar.pt")

/opt/miniconda3/lib/python3.9/site-packages/torch/serialization.py:602: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


### Evaluate performance of QAT model

In [25]:
print("Resnet QAT Profile:")
profile(qat_resnet)

Resnet QAT Profile:
Size (MB): 24.105327
Files already downloaded and verified
Files already downloaded and verified
Loss: 0.3617867136994998 
Accuracy: 0.8880208333333334
Time taken (1920 CIFAR test samples): 37.88538384437561


The accuracy of the QAT model is the highest, even slightly higher than the FP32 one! This is atypical, and is most likely because the CIFAR10 dataset is very simple for the Resnet architecture. Typically, we'd expect accuracies to drop from FP32-levels for more complex jobs.